### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [24]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [25]:
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("AIza"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [26]:
from openai import OpenAI

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)

MODEL = "gemini-2.5-flash"

In [27]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [28]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [29]:
def select_relevent_link(url):
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system", "content": link_system_prompt},
            {"role":"user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )

    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [30]:
select_relevent_link("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'associated company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [13]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gemini-2.5-flash
Found 11 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'related company/product', 'url': 'https://nebula.io/'},
  {'type': 'press release',
   'url': 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to gemini-2.0-flash-lite

In [15]:
def fetch_page_and_all_relevent_links(url):
    contetns = fetch_website_contents(url)
    relevent_links = select_relevant_links(url)
    result = f"##Lending page :\n\n {contetns}\n\n ## Relevent links:\n\n"
    for link in relevent_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link['url'])
    return result    

In [16]:
print(fetch_page_and_all_relevent_links("https://huggingface.com"))

Selecting relevant links for https://huggingface.com by calling gemini-2.5-flash
Found 17 relevant links
##Lending page :

 Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
6 days ago
•
269k
•
2.66k
microsoft/VibeVoice-Realtime-0.5B
Updated
1 day ago
•
120k
•
809
zai-org/GLM-4.6V-Flash
Updated
5 days ago
•
54.1k
•
397
mistralai/Devstral-Small-2-24B-Instruct-2512
Updated
2 days ago
•
13.7k
•
318
zai-org/GLM-4.6V
Updated
5 days ago
•
3.23k
•
292
Browse 1M+ models
Spaces
Running
on
Zero
540
Z Image Turbo
🖼
540
Generate stunning images from text prompts
Running
on
Zero
MCP
Featured
1.35k
Z Image Turbo
🏃
1.35k
Generate images from text prompts
Running
on
Zero
Featured

In [17]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevent_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [33]:
import time

def create_brochure(company_name, url):
    print(f"Creating brochure for {company_name} by calling {MODEL}")
    user_prompt = get_brochure_user_prompt(company_name, url)
    time.sleep(2)
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    brochure = response.choices[0].message.content
    display(Markdown(brochure))

In [34]:
create_brochure("Hugging Face", "https://huggingface.com")

Creating brochure for Hugging Face by calling gemini-2.5-flash
Selecting relevant links for https://huggingface.com by calling gemini-2.5-flash
Found 6 relevant links


# Hugging Face: The AI Community Building the Future

Hugging Face is the premier platform where the machine learning community collaborates on models, datasets, and applications. We are dedicated to accelerating AI development by providing an open, collaborative, and powerful environment for innovators worldwide. Our mission is to empower everyone, from individual practitioners to large enterprises, to create, discover, and build the future of artificial intelligence.

## Our Collaborative Platform

At the core of Hugging Face is a robust collaboration platform designed to make ML development faster, more accessible, and highly efficient. We offer:

*   **Models**: Explore and share over 1 million pre-trained models covering all modalities, including text, image, video, audio, and 3D.
*   **Datasets**: Access and contribute to over 250,000 datasets, perfect for training and fine-tuning your machine learning projects.
*   **Spaces**: Deploy and showcase your AI applications with over 400,000 interactive demos, making your innovations easily discoverable.
*   **Open Source Stack**: Leverage our comprehensive open-source tools to streamline your workflow and drive cutting-edge AI research and development.
*   **Build Your Portfolio**: Share your work with the world, build your professional ML profile, and connect with a global network of AI enthusiasts and experts.

## Accelerate Your ML Journey

Hugging Face offers tailored solutions to boost your personal projects or scale your organization's AI initiatives with advanced compute and enterprise-grade features.

### For Individuals: PRO Account

Elevate your personal Hugging Face experience for just **$9 per month**. With a PRO account, you'll receive:

*   10x private storage capacity
*   20x included inference credits
*   8x ZeroGPU quota and highest queue priority for compute
*   Spaces Dev Mode & ZeroGPU Spaces hosting
*   The ability to publish blog articles directly on your Hugging Face profile
*   Dataset Viewer functionality for private datasets
*   A distinctive Pro badge to showcase your support

### For Teams & Enterprises

Scale your organization's AI development with an advanced, secure, and collaborative platform tailored for professional environments.

*   **Team Subscription**: Get started instantly for growing teams at **$20 per user per month**.
*   **Enterprise Solutions**: For flexible contract options and custom requirements, please contact our sales team.

Our Team and Enterprise solutions enhance your Hugging Face experience with crucial features for organizational success:

*   **Advanced Security**: Benefit from Single Sign-On (SSO) integration, comprehensive Audit Logs, granular Resource Group access controls, and centralized Token Management.
*   **Data Control & Management**: Select and manage data residency with our Regions feature and enable the Dataset Viewer for your private datasets, facilitating easier collaboration.
*   **Enhanced Compute & Scalability**: Access advanced compute options like ZeroGPU with increased quotas and manage organization billing for Inference Providers.
*   **Private Storage**: Gain an additional 1 TB of private storage for each member of your organization (with options for more).
*   **Managed Billing**: Effectively control your budget with managed billing and flexible yearly commit options.
*   **Priority Support**: Maximize your platform usage with dedicated priority support from the Hugging Face team.

## Join the AI Community

Hugging Face is more than just a platform; it's a vibrant, global community dedicated to building the future of AI together. Whether you are exploring groundbreaking models, training powerful algorithms, or deploying innovative applications, you will find the tools, resources, and collaborative spirit to support your journey.

**Ready to start building the future of AI?** Sign up today or contact us to discover how Hugging Face can accelerate your AI ambitions.